## Goal 
 - Manually update the CSVs of a few stocks
 - Write a function to update a given CSV with that tickers data

----------------

 - Think of scenarios where this might fail, and what I can reasonably do to combat this
 - Write and schedule procedure to scrape for all **stock_data** stocks Monday-Friday after close
 - Write and schedule procedure to scrape for all S&P 500 stocks Monday-Friday after close

In [1]:
from stock_utils import *

In [2]:
from bs4 import BeautifulSoup
import requests
import re

### Yahoo

In [6]:
def update_CSV(path_to_csv, backup=False):
    if backup is True:
        raise ValueError('Not yet supported')
        
    ticker = ticker_from_csv(path_to_csv)
        
    ## Open Yahoo Finance data
    url = "https://finance.yahoo.com/quote/{sym}/history?p={sym}".format(sym=ticker)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    
    ## Store all rows of historical data for this stock in 'all_rows'
    all_rows = []
    historical_rows = soup.findAll('tr', attrs={'class':'BdT Bdc($c-fuji-grey-c) Ta(end) Fz(s) Whs(nw)'})
    for row in historical_rows:
        table_cells = row.findAll('td')
        row_list = []
        for cell in table_cells:
            #print(cell.text.strip())
            row_list.append(cell.text.strip())
        all_rows.append(row_list)
        
    ## Add any days that are missing to the CSV
    df = pd.DataFrame()
    df = df.from_csv(path_to_csv)
    
    for i in range(len(all_rows)):
        if pd.to_datetime(all_rows[i][0]) not in df.index:
            s_close = all_rows[i][5]
            s_volume = int(all_rows[i][6].replace(',', ''))
            s_open = all_rows[i][1]
            s_high = all_rows[i][2]
            s_low = all_rows[i][3]
            df.loc[pd.to_datetime(all_rows[i][0])] = [s_close, s_volume, s_open, s_high, s_low]
      
    df = df.sort_index(axis=0)
    df.to_csv(path_to_csv)

In [7]:
update_CSV('stock_data/amd.csv')

In [129]:
while True:
    symbol = input("Enter symbol: ")

    if symbol == 'stop':
        break    
    
    url = "https://finance.yahoo.com/quote/{sym}/history?p={sym}".format(sym=symbol)
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    
    all_rows = []
    historical_rows = soup.findAll('tr', attrs={'class':'BdT Bdc($c-fuji-grey-c) Ta(end) Fz(s) Whs(nw)'})
    for row in historical_rows:
        table_cells = row.findAll('td')
        row_list = []
        for cell in table_cells:
            #print(cell.text.strip())
            row_list.append(cell.text.strip())
        all_rows.append(row_list)

Enter symbol: CHTR
Enter symbol: stop


In [114]:
df = pd.DataFrame()
df = df.from_csv('stock_data/chtr.csv')
#df = df.sort_index(axis=0)

In [115]:
df.head()

,close,volume,open,high,low
date,,,,,
2017-09-22,360.30,2078451.0,359.54,363.33,358.750
2017-09-21,361.08,2374150.0,365.88,366.40,359.093
2017-09-20,367.01,2156612.0,365.99,371.50,363.500
2017-09-19,364.02,3293103.0,368.39,369.84,360.430
2017-09-18,368.17,1870865.0,370.20,371.51,365.900


Short-hand month name is **%b**   
Zero padded day is **%d**  
Fulle Year is **%Y**

In [116]:
sample_date = all_rows[7][0]

In [117]:
sample_date

'Oct 05, 2017'

In [118]:
#pd.to_datetime(sample_date)
converted_date = pd.to_datetime(sample_date).strftime('%Y/%m/%d')

In [119]:
pd.to_datetime(sample_date) == pd.to_datetime(converted_date)

True

In [120]:
for i in range(len(all_rows)):
    if pd.to_datetime(all_rows[i][0]) in df.index:
        print('in')
    else:
        #print(all_rows[i][0])
        s_close = all_rows[i][5]
        s_volume = int(all_rows[i][6].replace(',', ''))
        s_open = all_rows[i][1]
        s_high = all_rows[i][2]
        s_low = all_rows[i][3]
        df.loc[pd.to_datetime(all_rows[i][0])] = [s_close, s_volume, s_open, s_high, s_low]

in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in
in


In [121]:
df = df.sort_index(axis=0)

In [122]:
df.to_csv('stock_data/chtr.csv')

In [89]:
#df = df.from_csv('stock_data/chtr_temp.csv')

In [90]:
df.head()

,close,volume,open,high,low
date,,,,,
2016-04-12,221.4665,1213151.0,221.4775,223.6120,220.6370
2016-04-13,224.1871,1299353.0,223.4019,226.7087,221.1015
2016-04-14,222.0858,1178054.0,223.7890,224.5410,221.8646
2016-04-15,224.5742,1031018.0,223.4683,225.5917,220.3827
2016-04-18,222.9374,897291.0,223.2913,224.9060,222.3512


In [91]:
df.tail()

,close,volume,open,high,low
date,,,,,
2017-10-10,364.92,1111015.0,365.99,368.16,362.86
2017-10-11,365.12,1432621.0,364.06,366.97,361.03
2017-10-12,355.71,2592918.0,363.00,363.00,352.51
2017-10-13,194.59,15674800.0,193.55,195.00,191.65
2017-10-16,197.93,13972479.0,195.80,198.04,193.25


In [64]:
df.loc['2017-10-11']

close          190.94
volume    13237700.00
open           189.60
high           191.00
low            187.74
Name: 2017-10-11 00:00:00, dtype: float64

In [65]:
df.loc['2017-10-11']['volume']

13237700.0

In [66]:
df.loc['2017-10-11']['volume'] + 3

13237703.0